In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from kfsims.network import create_network, update_node_by_neighbors
from kfsims.common import init_trajectory
import numpy as np

In [3]:
traj = init_trajectory()

In [43]:
NODES = 30

# No fusion

In [44]:
# no fusion
def no_fusion_step_by_step():
    net = create_network(NODES)
    msrms = {node: (i for i in node.observe()) for node in net}
    rmses = []
    for i in range(traj.X.shape[1]):
        for node, ms in msrms.items():
            m = next(ms)
            node.single_kf(m)
    for node in net:
        rmses.append(node.post_rmse(traj.X))
    return np.mean(rmses)

In [45]:
no_fusion_step_by_step()

4.038209992211052

In [46]:
def no_fusion_all_at_once():
    # all at once, should be approx. same as the above
    # just a sanity check that the the step by step works
    # as expected since it's needed in diffusion
    net = create_network(NODES)
    rmses = []
    for node in net.nodes:
        node()
        rmses.append(node.post_rmse(traj.X))
    return np.mean(rmses)

In [47]:
no_fusion_all_at_once()

4.038209992211052

# Simple diffusion 

In [48]:
def simple_diffusion():
    net = create_network(NODES)
    msrms = {node: (i for i in node.observe()) for node in net}
    rmses = []
    for i in range(traj.X.shape[1]):

        # proveď jeden KF loop pro každý node
        for node, ms in msrms.items():
            m = next(ms)
            node.single_kf(m)

        # každému nodu přiřaď průměr hyperparametrů od všech sousedů
        for node in net:
            update_node_by_neighbors(net, node)


    for node in net:
        rmses.append(node.post_rmse(traj.X))
    return np.mean(rmses)

In [42]:
simple_diffusion()

2.6894649200582146